In [1]:
#Test suit for damage on interdependent, spatially imbedded networks
## Comparing independent and interdependent networks for flow and connectivity.

In [277]:
#Imports
# Standard imports
import copy
import itertools

# Scientific computing imports
import numpy
import matplotlib.pyplot as plt
from matplotlib import colors
import networkx as nx
import pandas as pd
import pickle
import collections
from datetime import datetime
from capacity_scaling import capacity_scaling 
from relabel_nodes import convert_node_labels_to_integers
from collections import Iterable
from collections import namedtuple
from bisect import bisect
from networkx.algorithms.flow import edmonds_karp
from networkx.algorithms.traversal.depth_first_search import dfs_tree
import networkx.algorithms.isomorphism as iso
from random import choice
from random import sample

In [255]:
#Damage Test

def add_demand(g,sys,components):
    """
    Adds component demand for system.
    """
    g=copy.deepcopy(g)
    for i in g:
        g.add_node(i,demand=0)
        
    for c,val in components[sys].iteritems():
        g.add_node(c,demand=val['d'])
    return g

def test_s(components,s_sol,sys):        
    """
    Test a single system for flow satisfaction
    """
    g=copy.deepcopy(s_sol)


    #Test flow
    cost,outflow=capacity_scaling(g)

    #Create inflow dict
    inflow={}
    for n in g.nodes():
        inflow[n]=0
    for pred,out in outflow.iteritems():
        for dest,in_f in out.iteritems():
            inflow[dest]+=in_f

    #Check demand satisfaction
    sat=1
    sat_dict={}
    for c,val in components[sys].iteritems():
        if val['d']>inflow[c]:
            sat=0
            sat_dict[c]=0
        else:
            sat_dict[c]=1

    return sat, inflow #, sat_dict

def test_s_connectivity(components,s_sol,sys):        
    """
    Test a single system for flow satisfaction
    """
    g=copy.deepcopy(s_sol)

    #Get source and sink information
    #s,t=get_s_t(sys,components)
    #print sys,s,t
    
    #Find nodes that are connected to working sinks
    supported=set() #list of all supported nodes
    for i in g:
        #print i, 'demand',g.node[i]['demand']
        if g.node[i]['demand']<0:
            supported.update([i])
            #print 'source',i,dfs_tree(s_sol,i).nodes()
            supported.update(dfs_tree(s_sol,i).nodes())
    #print supported
    
    #Check demand satisfaction
    sat=1
    sat_dict={}
    for c,val in components[sys].iteritems():
        if c in supported:
            sat=1
            sat_dict[c]=1
        else:
            sat=0
            sat_dict[c]=0

    return sat, sat_dict

def test_sos(base,components,sos):
    """
    Test system of system for flow satisfaction
    """
    #test_type =
    ## 'cc' - connectivity cross (interdependent)
    ## 'fc' - flow cross (interdependent)
    ## 'ci' - connectivity independent
    ## 'fi' - flow independent
    
    
    sos_eval=copy.deepcopy(sos)


    #Check component operating or not, failed if any demand not sat.
    comp_keys=[]
    for sys in components:
        comp_keys.extend(components[sys].keys())
    comp_keys=list(set(comp_keys))

    operating={k: 1 for k in comp_keys}        

    #Check if existing
    for sys,g in sos_eval.iteritems(): 
        for c,val in components[sys].iteritems():
            #print c, g.nodes(),val
            if (c not in g) and (val['d']!=0):
                #removed nodes are failed.
                operating[c]=0
            #g.add_node(c,demand=val['d'])      
    ini_op=copy.deepcopy(operating)
    
    #Add demand
    for sys,g in sos_eval.iteritems():
        sos_eval[sys]=add_demand(g,sys,components)
    
    #Test flow
    #Store SoS information        
    sos_info={k: {'sat':0,'inflow':{}} for k in components.keys()}
    op_list=[copy.deepcopy(operating)]
    while True:
        #Test individual systems 

        #print sos_eval
        for sys,g in sos_eval.iteritems():
            sat,inflow=test_s(components,g,sys)
            sos_info[sys]['sat']=sat
            sos_info[sys]['inflow']=inflow
            #sys_sat[sys]=sat_dict
        #print 'eval results', sos_info
        #check component dependence
        #failure=0


        for sys in components:
            for c,val in components[sys].iteritems():
                #what are links                    
                for l in val['l']:
                    #dependent system
                    d_sys=l[0]
                    threshold=l[1]

                    #check flow on dependent
                    dep_flow=sos_info[d_sys]['inflow'][c]
                    dep_req=components[d_sys][c]['d']
                    dep_ratio=float(dep_flow)/dep_req
                    #print 'link', c,l,'in=',dep_flow,'req=',dep_req
                    if dep_ratio<threshold:
                        #failure - remove demand from graph
                        sos_eval[sys].node[c]['demand']=0
                        operating[c]=0
                        #failure=1

        #store operating list
        op_list.append(copy.deepcopy(operating))
        #if no failure, converged. End while loop.
        if op_list[-1]==op_list[-2]: #failure==0
            #print 'operating list',op_list
            break
    fi=copy.deepcopy(op_list[1])
    fc=copy.deepcopy(operating)
    
    
    #Test connectivity
    #Add demand
    for sys,g in sos_eval.iteritems():
        sos_eval[sys]=add_demand(g,sys,components)
    
    #Store SoS information        
    sos_info={k: {'sat':0,'inflow':{}} for k in components.keys()}
    op_list=[copy.deepcopy(ini_op)]
    operating=copy.deepcopy(ini_op)
    while True:
        #Test individual systems 

        #print sos_eval
        for sys,g in sos_eval.iteritems():
            #print sys
            sat,inflow=test_s_connectivity(components,g,sys)
            #print inflow
            
            sos_info[sys]['sat']=sat
            sos_info[sys]['inflow']=inflow
            #sys_sat[sys]=sat_dict
        #print 'eval results', sos_info
        #check component dependence
        #failure=0


        for sys in components:
            for c,val in components[sys].iteritems():
                #what are links                    
                for l in val['l']:
                    #dependent system
                    d_sys=l[0]
                    threshold=l[1]

                    #check flow on dependent
                    dep_flow=sos_info[d_sys]['inflow'][c]
                    dep_req=components[d_sys][c]['d']
                    #dep_ratio=float(dep_flow)/dep_req
                    #print 'link', c,l,'in=',dep_flow,'req=',dep_req
                    if dep_req>0 and dep_flow==0:
                        #failure - remove demand from graph
                        sos_eval[sys].node[c]['demand']=0
                        operating[c]=0
                        #failure=1

        #store operating list
        op_list.append(copy.deepcopy(operating))
        #if no failure, converged. End while loop.
        if op_list[-1]==op_list[-2]: #failure==0
            #print 'operating list',op_list
            break
    #print op_list

    ci=copy.deepcopy(op_list[1])
    cc=copy.deepcopy(operating)

    return fi, fc, ci, cc


def get_s_t(sys,components):
    """
    Gets source and sink info for system.
    """
    s_dict={}
    t_dict={}

    for c,info in components[sys].iteritems():
        if info['d']<0:
            s_dict[c]=-info['d']

        if info['d']>0:
            t_dict[c]=info['d']

    return s_dict,t_dict

def score_sos(v,c,sos,tests,n_rem=1,blast=1):
        """
        Get score of system of systems
        """
        
        #Survivability
        results={}
        results['fi']=[]
        results['fc']=[]
        results['ci']=[]
        results['cc']=[]
        
        op={}
        op['fi']=[]
        op['fc']=[]
        op['ci']=[]
        op['cc']=[]
        
        
        for i in xrange(tests):
            sos_damage=copy.deepcopy(sos)
            #remove nodes due to damage
            sos_damage=inflict_damage(v,sos_damage,n_rem,blast)
            
            #for sys in sos_damage:
                #print sys,sos_damage[sys].nodes(data=True),sos_damage[sys].edges(data=True)
            
            #Test flow            
            fi,fc,ci,cc=test_sos(v,c,sos_damage)
            
            #flow
            #no-cross
            working=sum(fi.itervalues())
            results['fi'].append(float(working)/len(fi.keys()))
            op['fi'].append(fi)
            
            #cross
            working=sum(fc.itervalues())
            results['fc'].append(float(working)/len(fc.keys()))
            op['fc'].append(fc)
            
            #connectivity
            #no-cross
            working=sum(ci.itervalues())
            results['ci'].append(float(working)/len(ci.keys()))
            op['ci'].append(ci)
            
            #cross
            working=sum(cc.itervalues())
            results['cc'].append(float(working)/len(cc.keys()))
            op['cc'].append(cc)
            
        score={}
        for t in results:
            score[t]=numpy.average(results[t])
        #print results,s_score

        
        return score,results,op

In [69]:
#Testing - ship

#Define vessel
area = nx.grid_graph(dim=[8,6])
area.remove_node((0,0))
area.remove_node((0,5))
area.remove_node((0,4))
area.remove_node((1,5))
area.remove_node((1,4))
area.remove_node((4,5))
area.remove_node((4,4))
area.remove_node((5,5))
area.remove_node((5,4))
area.remove_node((6,5))
area.remove_node((6,4))
area.remove_node((7,5))
area.remove_node((7,4))



#Define components
c={1:{(5,3):{'d':1,'l':[(1,1.0)]},
      (3,3):{'d':1,'l':[(1,1.0)]},
      (1,0):{'d':1,'l':[(1,1.0)]},
      (1,2):{'d':1,'l':[(1,1.0)]},
      (3,5):{'d':1,'l':[(1,1.0)]},
      (4,0):{'d':-6,'l':[(2,1.0)]},
      (5,0):{'d':1,'l':[(1,1.0)]}},
   2:{(5,3):{'d':1,'l':[(2,1.0)]},
      (3,3):{'d':1,'l':[(2,1.0)]},
      (1,0):{'d':1,'l':[(2,1.0)]},
      (1,2):{'d':1,'l':[(2,1.0)]},
      (3,5):{'d':1,'l':[(2,1.0)]},
      (4,0):{'d':1,'l':[(2,1.0)]},
      (5,0):{'d':-6,'l':[(1,1.0)]}}}

#Define SoS
sos={}

g=nx.DiGraph()
g.add_edges_from([((3, 0), (3, 1), {'capacity': 6}),
 ((2, 1), (1, 1), {'capacity': 1}),
 ((2, 1), (2, 2), {'capacity': 1}),
 ((5, 1), (5, 2), {'capacity': 3}),
 ((4, 0), (3, 0), {'capacity': 3}),
 ((4, 0), (4, 1), {'capacity': 6}),
 ((4, 0), (5, 0), {'capacity': 3}),
 ((3, 3), (3, 4), {'capacity': 3}),
 ((2, 2), (1, 2), {'capacity': 3}),
 ((4, 1), (5, 1), {'capacity': 3}),
 ((4, 1), (4, 2), {'capacity': 3}),
 ((1, 1), (1, 0), {'capacity': 1}),
 ((4, 2), (4, 3), {'capacity': 6}),
 ((3, 4), (3, 5), {'capacity': 1}),
 ((3, 1), (2, 1), {'capacity': 3}),
 ((4, 3), (3, 3), {'capacity': 6}),
 ((5, 2), (5, 3), {'capacity': 1})])

sos[1]=copy.deepcopy(g)

g=nx.DiGraph()
g.add_edges_from([((2, 1), (2, 0), {'capacity': 1}),
 ((5, 1), (4, 1), {'capacity': 6}),
 ((5, 1), (5, 2), {'capacity': 3}),
 ((3, 3), (3, 4), {'capacity': 3}),
 ((5, 0), (5, 1), {'capacity': 6}),
 ((5, 0), (4, 0), {'capacity': 3}),
 ((2, 2), (1, 2), {'capacity': 1}),
 ((4, 1), (3, 1), {'capacity': 3}),
 ((3, 2), (3, 3), {'capacity': 1}),
 ((3, 2), (2, 2), {'capacity': 6}),
 ((4, 2), (3, 2), {'capacity': 1}),
 ((4, 2), (4, 3), {'capacity': 1}),
 ((5, 2), (4, 2), {'capacity': 6}),
 ((5, 2), (5, 3), {'capacity': 1}),
 ((3, 1), (3, 2), {'capacity': 3}),
 ((3, 1), (2, 1), {'capacity': 3}),
 ((2, 0), (1, 0), {'capacity': 1}),
 ((4, 3), (3, 3), {'capacity': 1}),
 ((3, 4), (3, 5), {'capacity': 3})])

sos[2]=copy.deepcopy(g)


#1->b blasts
#test=50

In [283]:
#Testing -simple

#Define vessel
area = nx.grid_graph(dim=[2,2])

#Define components
c={1:{(0,1):{'d':-1,'l':[(2,1.0)]},
      (0,0):{'d':-1,'l':[(2,1.0)]},
      (1,0):{'d':1,'l':[(1,1.0)]}},
   2:{(0,1):{'d':1,'l':[(2,1.0)]},
      (0,0):{'d':1,'l':[(2,1.0)]},
      (1,0):{'d':-2,'l':[(1,1.0)]}}}

#Define SoS
sos={}

g=nx.DiGraph()
g.add_edges_from([((0, 1), (1, 1), {'capacity': 1}),
                  ((0, 0), (1, 0), {'capacity': 1}),
                  ((1, 1), (1, 0), {'capacity': 1})])

sos[1]=copy.deepcopy(g)

g=nx.DiGraph()
g.add_edges_from([((1, 0), (0, 0), {'capacity': 2}),
                  ((0, 0), (0, 1), {'capacity': 1})])

sos[2]=copy.deepcopy(g)

In [313]:
#Testing - flow vs connectivity

#Define vessel
area = nx.grid_graph(dim=[2,2])

#Define components
c={1:{(0,1):{'d':-2,'l':[(2,1.0)]},
      (1,0):{'d':2,'l':[(1,1.0)]}},
   2:{(0,1):{'d':1,'l':[(2,1.0)]},
      (1,0):{'d':-1,'l':[(1,1.0)]}}}

#Define SoS
sos={}

g=nx.DiGraph()
g.add_edges_from([((0, 1), (1, 1), {'capacity': 1}),
                  ((0, 0), (1, 0), {'capacity': 1}),
                  ((0, 1), (0, 0), {'capacity': 1}),
                  ((1, 1), (1, 0), {'capacity': 1})])

sos[1]=copy.deepcopy(g)

g=nx.DiGraph()
g.add_edges_from([((1, 0), (0, 0), {'capacity': 1}),
                  ((0, 0), (0, 1), {'capacity': 1})])

sos[2]=copy.deepcopy(g)

In [261]:
#Testing - 4 sys

#Define vessel
area = nx.grid_graph(dim=[2,2])

#Define components
c={1:{(0,1):{'d':-1,'l':[(4,1.0)]},
      (0,0):{'d':0,'l':[]},
      (1,0):{'d':0,'l':[]},
      (1,1):{'d':1,'l':[(1,1.0)]}},
   2:{(0,1):{'d':0,'l':[]},
      (0,0):{'d':0,'l':[]},
      (1,0):{'d':1,'l':[(2,1.0)]},
      (1,1):{'d':-1,'l':[(1,1.0)]}},
   3:{(0,1):{'d':0,'l':[]},
      (0,0):{'d':1,'l':[(3,1.0)]},
      (1,0):{'d':-1,'l':[(2,1.0)]},
      (1,1):{'d':0,'l':[]}},
   4:{(0,1):{'d':1,'l':[(4,1.0)]},
      (0,0):{'d':-1,'l':[(3,1.0)]},
      (1,0):{'d':0,'l':[]},
      (1,1):{'d':0,'l':[]}}
  }

#Define SoS
sos={}

g=nx.DiGraph()
g.add_edges_from([((0, 1), (1, 1), {'capacity': 1})])
print g.edges(data=True)
sos[1]=copy.deepcopy(g)

g=nx.DiGraph()
g.add_edges_from([((1, 1), (1, 0), {'capacity': 1})])
sos[2]=copy.deepcopy(g)

g=nx.DiGraph()
g.add_edges_from([((1, 0), (0, 0), {'capacity': 1})])
sos[3]=copy.deepcopy(g)

g=nx.DiGraph()
g.add_edges_from([((0,0 ), (0, 1), {'capacity': 1})])
sos[4]=copy.deepcopy(g)

[((0, 1), (1, 1), {'capacity': 1})]


In [310]:
#Damage inflict, blast radius=1


def inflict_damage(v,sos,n_rem,blast):
    """
    Removes nodes from sos.
    """
    targets=sample(v.nodes(),
                           n_rem)
    #print 'removed', nodes_to_remove
    #Get radius
    nodes_to_remove=set()
    #print 'num', n_rem
    
    #targets=[(0,0)]

    nodes_to_remove.update(targets)
    #nodes_to_remove.update([(0,0)])
    print nodes_to_remove

    for n in targets:
        path=nx.single_source_shortest_path(v,
                                            n,
                                            cutoff=blast)
        for t in path:
            nodes_to_remove.update(path[t])

    #print 'final',nodes_to_remove


    #Remove node in each system
    #print nodes_to_remove
    for sys,g in sos.iteritems():
        for n in nodes_to_remove:

            if n in g:
                g.remove_node(n)

    return sos

In [303]:
def testing(area,c,sos,tests,number_of_removals,radius):

    s,r,o=score_sos(area,c,sos,tests,n_rem=number_of_removals,blast=radius)
    
    component_names=[]
    for sys,comps in c.iteritems():
        for comp in comps:
            component_names.append(comp)

    component_names=list(set(component_names))

    data=[]

    for a_type in r:
        for trial in zip(r[a_type],o[a_type]):
            operating=[]
            for comp in component_names:
                operating.append(trial[1][comp])
            #print operating
            #print trial[0]
            trial_data=list(itertools.chain([n_rem,blast],[a_type],[trial[0]],operating))
            data.append(trial_data)

    cs=list(itertools.chain(['hits','blast','eval','score'],component_names))

    d=pd.DataFrame(data,columns=cs)

    return d

In [308]:
#Define vessel
area = nx.grid_graph(dim=[2,2])

#Define components
c={1:{(0,1):{'d':-1,'l':[(2,1.0)]},
      (0,0):{'d':-1,'l':[(2,1.0)]},
      (1,0):{'d':1,'l':[(1,1.0)]}},
   2:{(0,1):{'d':1,'l':[(2,1.0)]},
      (0,0):{'d':1,'l':[(2,1.0)]},
      (1,0):{'d':-2,'l':[(1,1.0)]}}}

#Define SoS
sos={}

g=nx.DiGraph()
g.add_edges_from([((0, 1), (1, 1), {'capacity': 1}),
                  ((0, 0), (1, 0), {'capacity': 1}),
                  ((1, 1), (1, 0), {'capacity': 1})])

sos[1]=copy.deepcopy(g)

g=nx.DiGraph()
g.add_edges_from([((1, 0), (0, 0), {'capacity': 2}),
                  ((0, 0), (0, 1), {'capacity': 1})])

sos[2]=copy.deepcopy(g)

data=testing(area,c,sos,tests=10,number_of_removals=1,radius=0)
print data

    hits  blast eval     score  (0, 1)  (1, 0)  (0, 0)
0      1      0   cc  0.000000       0       0       0
1      1      0   cc  0.000000       0       0       0
2      1      0   cc  1.000000       1       1       1
3      1      0   cc  0.666667       0       1       1
4      1      0   cc  0.000000       0       0       0
5      1      0   cc  0.000000       0       0       0
6      1      0   cc  0.666667       0       1       1
7      1      0   cc  0.666667       0       1       1
8      1      0   cc  0.666667       0       1       1
9      1      0   cc  1.000000       1       1       1
10     1      0   fi  0.000000       0       0       0
11     1      0   fi  0.000000       0       0       0
12     1      0   fi  0.666667       0       1       1
13     1      0   fi  0.666667       0       1       1
14     1      0   fi  0.333333       0       1       0
15     1      0   fi  0.000000       0       0       0
16     1      0   fi  0.666667       0       1       1
17     1  

In [315]:
n_rem=1
blast=0
s,r,o=score_sos(area,c,sos,1,n_rem=n_rem,blast=blast)

print 'score:'
for t in s:
    print t,s[t]
    
print 'results:'
for t in r:
    print t,r[t]

print 'operation:'
for t in o:
    print t,o[t]
    
test=list(itertools.chain(r,o))

set([(0, 1)])
score:
cc 0.0
fi 0.0
ci 0.0
fc 0.0
results:
cc [0.0]
fi [0.0]
ci [0.0]
fc [0.0]
operation:
cc [{(0, 1): 0, (1, 0): 0}]
fi [{(0, 1): 0, (1, 0): 0}]
ci [{(0, 1): 0, (1, 0): 0}]
fc [{(0, 1): 0, (1, 0): 0}]


In [286]:
component_names=[]
for sys,comps in c.iteritems():
    for comp in comps:
        component_names.append(comp)
        
component_names=set(component_names)
    

cs=list(itertools.chain(['hits','blast','eval','score'],component_names))
print cs

df=pd.DataFrame(data=d,index=1)

print df

['hits', 'blast', 'eval', 'score', (0, 1), (1, 0)]


TypeError: Index(...) must be called with a collection of some kind, 1 was passed

In [327]:
sos[1].add_edge((0,0),(0,1),capacity=1)
print sos[1].edges(data=True)

s=nx.Graph()
s.add_edges_from(sos[1].edges())
print s.edges(data=True)

[((0, 1), (0, 0), {'capacity': 1}), ((0, 1), (1, 1), {'capacity': 1}), ((0, 0), (0, 1), {'capacity': 1}), ((0, 0), (1, 0), {'capacity': 1}), ((1, 1), (1, 0), {'capacity': 1})]
[((0, 1), (0, 0), {}), ((0, 1), (1, 1), {}), ((1, 0), (0, 0), {}), ((1, 0), (1, 1), {})]


In [330]:
g=nx.DiGraph()
g.add_edge(1,2)
print dfs_tree(g,2).nodes()

g=nx.Graph()
g.add_edge(1,2)
print dfs_tree(g,2).nodes()

[2]
[1, 2]
